In [14]:
import os
from IPython.display import display, Image, clear_output
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import json


import sys

sys.path.append("/src/")
from visualization.visualize import (
    print_image_with_point,
    show_complete_fixation_with_all_frames_all_gaze,
)

In [15]:
folder_path = "path/to/your/folder/"
# load from 
eval_results_path = folder_path + "data/cyprus_eval_frames_results.csv"
prediction_path =  folder_path + "data/fixation_and_labels_total.csv"
# save to
cyprus_eval_frames_results_predictions = folder_path + "cyprus_eval_frames_results_predictions.csv"

# load the names and print the categories
mapping_path = folder_path + "/label_mapping.json"
with open(mapping_path, "r") as f:
    label_mapping = json.load(f)
    categories = set(label_mapping["category_mapping"].values())

In [16]:
# combine the manual labels with the predictions
fix_df = pd.read_csv(prediction_path)
eval_df = pd.read_csv(eval_results_path)

newdf_list = []
for i, row in eval_df.iterrows():
    session = row["session"]
    frame_nr = row["frame_nr"]
    response = row["response"]
    x = row["x"]
    y = row["y"]
    str_frame_nr = f"{frame_nr:05d}"
    imgs_path = folder_path + session + "/video_frames_img/"

    fix_id = fix_df[
        (fix_df["frame_nr"] == frame_nr) & (fix_df["session"] == session)
    ].iloc[0]["fixation_id"]

    predictedsum = fix_df[
        (fix_df["fixation_id"] == fix_id) & (fix_df["session"] == session)
    ].iloc[0]["fixation_sum_label"]
    levelsum = fix_df[
        (fix_df["fixation_id"] == fix_id) & (fix_df["session"] == session)
    ].iloc[0]["level_sum_annotation"]
    predictedpXc = fix_df[
        (fix_df["fixation_id"] == fix_id) & (fix_df["session"] == session)
    ].iloc[0]["fixation_pXc_label"]
    levelpXc = fix_df[
        (fix_df["fixation_id"] == fix_id) & (fix_df["session"] == session)
    ].iloc[0]["level_pXc_annotation"]

    newrow = [
        session,
        frame_nr,
        fix_id,
        x,
        y,
        response,
        levelsum,
        predictedsum,
        levelpXc,
        predictedpXc,
    ]
    newdf_list.append(newrow)


# list of list to dataframe
newdf = pd.DataFrame(
    newdf_list,
    columns=[
        "session",
        "frame_nr",
        "fixation_id",
        "x",
        "y",
        "response",
        "level_sum",
        "predicted_sum",
        "level_pXc",
        "predicted_pXc",
    ],
)


In [18]:
# store the new dataframe
newdf.to_csv(cyprus_eval_frames_results_predictions, index=False)

In [ ]:
### show results - just for testing

eval_path_results_prediction = folder_path + "cyprus_eval_frames_results_prediction.csv"
data = pd.read_csv(eval_path_results_prediction)
# Add an empty 'response' column to your DataFrame if it doesn't already exist

counter = 0
for i, row in data.iterrows():


    # add some properties so that not all frames are shown
    if row["response"] == row["predicted"]:
        continue
    elif not (row["response"] == "ambiguous" and row["level"] != "LEVEL_4"):
        continue
    counter += 1


    ## get frame number
    x = row["x"]
    y = row["y"]
    session = row["session"]
    frame_nr = row["frame_nr"]

    str_frame_nr = f"{frame_nr:05d}"
    imgs_path = folder_path + session + "/video_frames_img/"
    label_fixation_path = folder_path + session + "/fixation_and_labels_leveled.csv"

    fix_df = pd.read_csv(label_fixation_path)
    fix_id = fix_df[fix_df["frame_nr"] == frame_nr].iloc[0]["fixation_id"]
    predicted = fix_df[fix_df["fixation_id"] == fix_id].iloc[0][
        "fix_annotation_max_sum"
    ]
    level = fix_df[fix_df["fixation_id"] == fix_id].iloc[0]["level_sum_annotation"]

    response = row["response"]
    predicte = row["predicted"]

    print(
        f"Frame {i+1} of {len(data)}\n{session},{str_frame_nr},{response}, {predicte}"
    )
    try:
        show_complete_fixation_with_all_frames_all_gaze(fix_df, fix_id, imgs_path)
    except:
        print("Error wit frame", frame_nr)